In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import joblib
import pandas as pd
import numpy as np
import faiss
sys.path.append('../')

os.environ['TRANSFORMERS_CACHE'] = '/data/users/kartik/hf_cache/'
os.environ['CUDA_VISIBLE_DEVICES']='0'

from prod_code.get_predictions import GetPredictions



2022-08-05 21:11:07.008265: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Seed set to 42


In [15]:
load_path='/data/users/kartik/Dictionary_and_Word_Embeddings/checkpoints/xlm-roberta-large_bert-base-uncased_0'


In [3]:
from utils.faiss_utils import create_and_store_index, get_top_n_accuracy
from trainer.params import get_roberta_params, get_xlm_params
import json
params = json.load(open(load_path+'/params.json'))
create_and_store_index(
    params['emb_type'], params['knn_measure']
    )


Loading Data for Indexing...
loading files: ['../data/clean_data/val.joblib', '../data/clean_data/test.joblib']
Creating FAISS Index...
Data len before unique_mapping: 8950
Data len after unique_mapping: 8950
Index shape (8950, 768)
FAISS Index Creation Done...


In [6]:
params

{'model_checkpoint': 'xlm-roberta-large',
 'source_column': 'gloss',
 'max_len': 150,
 'batch_size': 16,
 'dropout': 0.1,
 'learning_rate': 0.0001,
 'num_epochs': 150,
 'early_stopping_limit': 5,
 'device': 'cuda:0',
 'loss_fn_name': 'cosine',
 'emb_type': 'bert-base-uncased',
 'use_adapters': True,
 'resume_from_checkpoint': False,
 'output_size': 768,
 'knn_measure': 'cosine',
 'num_warmup_steps': 300,
 'extra_data': False}

In [18]:
getpred = GetPredictions(save_checkpoint_path=load_path, model_type='multilingual') #mono/multi


Preparing Model and Search Attributes...

Loading Tokenizer: xlm-roberta-large ...
Loading from : /data/users/kartik/Dictionary_and_Word_Embeddings/checkpoints/xlm-roberta-large_bert-base-uncased_0/model.pt

Using device cuda:0 for training...


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading Checkpoint ...
Setting Models state to checkpoint...
Model state set.
Index dict len: 8950


In [19]:
val = joblib.load('../data/clean_data/val.joblib')
ranks, nearest = getpred.get_predictions(val, NEAREST_NEIGHBOUR_NUM = 1000, pred_batch = 64) #

  0%|          | 0/71 [00:00<?, ?it/s]

Average Rank @1000 : 292.3144432194046
Median Rank @1000 : 86.0
Rank Variance @1000 : 370.52913015895905
Top 1 accuracy is 0.074
Top 2 accuracy is 0.11
Top 5 accuracy is 0.168
Top 10 accuracy is 0.228
Top 50 accuracy is 0.421
Top 100 accuracy is 0.523


In [11]:
# val

In [39]:
val = joblib.load('../data/clean_data/val.joblib')
# val = joblib.load('../data/clean_data/de_wiki_muse.joblib')
ranks, nearest = getpred.get_predictions(val, NEAREST_NEIGHBOUR_NUM = 1000, pred_batch = 64) #

  0%|          | 0/66 [00:00<?, ?it/s]

Average Rank @1000 : 158.88196255036738
Median Rank @1000 : 16.0
Rank Variance @1000 : 296.1288889106231
Top 1 accuracy is 0.168
Top 2 accuracy is 0.242
Top 5 accuracy is 0.353
Top 10 accuracy is 0.443
Top 50 accuracy is 0.645
Top 100 accuracy is 0.723


In [14]:
getpred.get_prediction_on_sentence("आराम की स्थिति में।")

['footbath',
 'slackish',
 'sleepily',
 'sideboard',
 'heavy-hearted',
 'angst',
 'poise',
 'weak',
 'worn-out',
 'pleasurable',
 'prescient',
 'skiddly',
 'galvanic',
 'up',
 'hunch',
 'knees-up',
 'mellow',
 'bespectacled',
 'blubbery',
 'egg-crate',
 'cabined',
 'roughcast',
 'passionless',
 'gentle',
 'cavort',
 'ankle-deep',
 'world-weary',
 'deckhand',
 'belt',
 'brazen',
 'languish',
 'prestidigitation',
 'cosseted',
 'graceless',
 'tensen',
 'craggy',
 'legroom',
 'confusional',
 'transitorily',
 'sedentary',
 'resolute',
 'emptyhanded',
 'onside',
 'swimmy',
 'stiffness']